In [1]:
import tensorflow as tf
import numpy as np
from sklearn import datasets

In [5]:
def get_batch(X, size):
    a = np.random.choice(len(X), size, replace=False)
    return X[a]      

In [8]:
class Autoencoder:
    def __init__(self, input_dim, hidden_dim, epoch=250, learning_rate=0.001):
        self.epoch = epoch
        self.learning_rate = learning_rate
        x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim])
        with tf.name_scope('encode'):
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([hidden_dim]), name='biases')
            encoded = tf.nn.tanh(tf.matmul(x, weights) + biases)
        with tf.name_scope('decode'):
            weights = tf.Variable(tf.random_normal([hidden_dim, input_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([input_dim]), name='biases')
            decoded = tf.matmul(encoded, weights) + biases
        
        self.x = x
        self.encoded = encoded
        self.decoded = decoded
        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoded))))
        self.train_op = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)
        self.saver = tf.train.Saver()
        
    def train(self, data):
        num_samples = len(data)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                for j in range(num_samples):
                    l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x: [data[j]]})
                if i % 10 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l))
                    self.saver.save(sess, './model.ckpt')
            self.saver.save(sess, './model.ckpt')
            
    def test(self, data): 
        with tf.Session() as sess:
            self.saver.restore(sess, './model.ckpt')
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict={self.x: data})
        print('input', data)
        print('compressed', hidden)
        print('reconstructed', reconstructed)
        return reconstructed

    def train(self, data, batch_size=10):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                for j in range(500):
                    batch_data = get_batch(data, batch_size)
                    l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x: batch_data})
                if i % 10 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l))
                    self.saver.save(sess, './model.ckpt')
            self.saver.save(sess, './model.ckpt')

In [9]:
hidden_dim = 1
data = datasets.load_iris().data
input_dim = len(data[0])
ae = Autoencoder(input_dim, hidden_dim)
ae.train(data)
ae.test([[8, 4, 6, 2]])

epoch 0: loss = 3.4875733852386475
epoch 10: loss = 0.9527798295021057
epoch 20: loss = 0.36293140053749084
epoch 30: loss = 0.30661630630493164
epoch 40: loss = 0.32359886169433594
epoch 50: loss = 0.29673701524734497
epoch 60: loss = 0.23335884511470795
epoch 70: loss = 0.288099467754364
epoch 80: loss = 0.329174280166626
epoch 90: loss = 0.3314952254295349
epoch 100: loss = 0.22875824570655823
epoch 110: loss = 0.24849380552768707
epoch 120: loss = 0.3396032154560089
epoch 130: loss = 0.3187870383262634
epoch 140: loss = 0.36837998032569885
epoch 150: loss = 0.2701427638530731
epoch 160: loss = 0.3142606019973755
epoch 170: loss = 0.3239821493625641
epoch 180: loss = 0.2427857369184494
epoch 190: loss = 0.36030280590057373
epoch 200: loss = 0.23264630138874054
epoch 210: loss = 0.20133259892463684
epoch 220: loss = 0.25987690687179565
epoch 230: loss = 0.22541143000125885
epoch 240: loss = 0.3141627311706543
Instructions for updating:
Use standard file APIs to check for files with t

array([[6.90891  , 2.8226993, 6.2510552, 2.2447534]], dtype=float32)